In [1]:
# Sentiment Analysis
import nltk
from nltk.corpus import movie_reviews

In [ ]:
dir(movie_reviews) #categories, words and fileids

In [2]:
# list of words for all files in movie_reviews with category
documents = []

for category in movie_reviews.categories():
    for fileid in movie_reviews.fileids(category):
        documents.append((list(movie_reviews.words(fileid)), category))

In [3]:
import random

random.shuffle(documents)
# print(documents[1][0])

In [4]:
from nltk.probability import FreqDist #FreqDist arranges the words in descendng order of frequently appeared words.
all_words = []

for word in movie_reviews.words():
    all_words.append(word.lower())
    
all_words = FreqDist(all_words)
word_features = list(all_words)[:3000] #Most frequent words in movie_words

In [5]:
# Relation of documents with features(if present or not)
def find_features(document):
    words = set(document)
    features = {}
    for w in word_features:
        features[w] = (w in words)

    return features

In [6]:
# print((find_features(movie_reviews.words('neg/cv000_29416.txt'))))

#Prepare dataset
featuresets=[]
for (rev, category) in documents:
    featuresets.append((find_features(rev), category))
    
print(len(featuresets))

2000


In [7]:
# set that we'll train our classifier with
training_set = featuresets[:1900]

# set that we'll test against.
testing_set = featuresets[1900:] #100 sets

In [8]:
classifier = nltk.NaiveBayesClassifier.train(training_set)

In [9]:
import pickle

#Dump the classifier
classifier_dump = open('classifier.pickle', 'wb')
new_classifier = pickle.dump(classifier, classifier_dump)
classifier_dump.close()

#Load the classifier
classifier_f = open('classifier.pickle', 'rb')
classifier = pickle.load(classifier_f)
classifier_f.close()

In [11]:
print("Classifier accuracy percent:",(nltk.classify.accuracy(classifier, testing_set))*100)
# classifier.show_most_informative_features(15)

Classifier accuracy percent: 87.0
